# 基于深度学习的文本分类 1

## 1. 文本表示方法

### 1.1 现有文本表示方法的缺陷

上一章节介绍过的几种文本表示方法：
1. One-hot;
2. Bag of Words;
3. N-gram;
4. TF-IDF;

+ 上述方法存在的问题：转换得到的向量维度很高，需要较长的训练实践；没有考虑单词与单词之间的关系，只是进行了统计;
+ 与上述方法不同，深度学习也可以用于文本表示，还可以将其映射到一个低纬空间. 其中比较典型的例子有：FastText、Word2Vec 和 Bert.

## 2. FastText

### 2.1 FastText 介绍

+ FastText是一种典型的深度学习词向量的表示方法，它通过 Embedding 层将单词映射到稠密空间，然后将句子中所有的单词在 Embedding 空间中进行平均，进而完成分类操作.
+ FastText 是一个三层的神经网络，输入层、隐含层和输出层;

In [1]:
from __future__ import unicode_literals

In [2]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.layers import Dense

Using TensorFlow backend.
E:\Anaconda\Anaconda\envs\py3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda\Anaconda\envs\py3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda\Anaconda\envs\py3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda\Anaconda\envs\py3.6\lib\site-packages\tensorflo

In [3]:
VOCAB_SIZE = 2000
EMBEDDING_DIM = 100
MAX_WORDS = 500
CLASS_NUM = 5

In [4]:
def build_fastText():
    model = Sequential()
    # 通过embedding层，将词汇映射成EMBEDDING_DIM维向量
    model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM,input_length=MAX_WORDS))
    # 通过GlobalAveragePooling1D平均了文档中所有词的embedding
    model.add(GlobalAveragePooling1D())
    # 通过输出层softmax分类，得到类别概率分布
    model.add(Dense(CLASS_NUM, activation='softmax'))
    # 定义损失函数、优化器、分类度量指标
    model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
    return model

In [5]:
if __name__ == "__main__":
    model = build_fastText()
    print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          200000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 505       
Total params: 200,505
Trainable params: 200,505
Non-trainable params: 0
_________________________________________________________________
None


### 2.2 FastText 优点

+ FastText 用单词的 Embedding 叠加获得的文档向量，将相似的句子分为一类;
+ FastText 学习到的 Embedding 空间维度比较低，可以快速进行训练;

## 3. 基于 FastText 的文本分类

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
import fasttext

# 转换FastText需要的格式
train_df = pd.read_csv('./data/train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text', 'label_ft']].iloc[:-5000].to_csv('train.csv', index=None, header=None, sep='\t')

model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
verbose=2, minCount=1, epoch=25, loss='hs')

val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
print(f1_score(train_df['label'].values[-5000:].astype(str),val_pred, average='macro'))

## 4. 使用验证集调参

In [ ]:
label2id = {}
for i in range(total):
    label = str(all_labels[i])
    if label not in label2id:
        label2id[label] = [i]
    else:
        label2id[label].append(i)